In [207]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta

In [208]:
def nofluffjobs_page_job_offers() -> dict:

    nofluffjobs_dict = dict()
    id_counter = 1

    url = "https://nofluffjobs.com/pl/praca-zdalna/python?criteria=city%3Dwarszawa%20seniority%3Dtrainee,junior%20%20salary<pln12000m&page=1"

    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find("div", class_="list-container ng-star-inserted")
    
    for job_element in soup.select('a[class*="posting-list-item posting-list-item--"]'):
        
        
        
        company_element = job_element.find("span", class_="d-block posting-title__company text-truncate")
        job_title_element = job_element.find("h3", class_="posting-title__position text-truncate color-main ng-star-inserted")
        if "Fullstack" not in job_title_element.text and "DevOps" not in job_title_element.text:
            link_url = "http://nofluffjobs.com" + job_element["href"]
            page_job_element = requests.get(link_url)
            soup_page = BeautifulSoup(page_job_element.content, "html.parser")
            position = soup_page.find("span", class_="mr-10 font-weight-medium")
            publication_date_element = soup_page.find("div", class_="posting-time-row")
            days_after_publication = int(re.findall(r'\b\d+\b', publication_date_element.text.strip())[0])
            publication_date = (datetime.today() - (timedelta(days=days_after_publication))).strftime("%Y-%m-%d")
            #creation of job offers details dictionary
            nofluffjobs_dict[id_counter] = {}
            nofluffjobs_dict[id_counter]["publication_date"] = publication_date
            nofluffjobs_dict[id_counter]["company"] = company_element.text.strip()
            nofluffjobs_dict[id_counter]["title"] = job_title_element.text.strip()
            nofluffjobs_dict[id_counter]["position"] = position.text.strip()
            nofluffjobs_dict[id_counter]["link_url"] = link_url

            id_counter += 1
    return nofluffjobs_dict


In [209]:
import pandas as pd
import numpy as np

nofluffjobs_dict = nofluffjobs_page_job_offers()
df_raw = pd.DataFrame.from_dict(nofluffjobs_dict, orient="index")
df = df_raw.copy()
df['publication_date'] = pd.to_datetime(df['publication_date'], infer_datetime_format=True)
df.drop_duplicates(subset=['publication_date', 'company', 'title'], inplace=True, ignore_index=True)


In [206]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   publication_date  7 non-null      datetime64[ns]
 1   company           7 non-null      object        
 2   title             7 non-null      object        
 3   position          7 non-null      object        
 4   link_url          7 non-null      object        
dtypes: datetime64[ns](1), object(4)
memory usage: 408.0+ bytes
